Here I want to explore how to evaluate expectation values in Qiskit, to analyze the VQE results  
Most of the stuff is explained here https://qiskit.org/documentation/tutorials/operators/01_operator_flow.html  
These issue tickets are somewhat also helpful  
[Inconsistence between qasm and eval outputs of opflow expectation value #6255](https://github.com/Qiskit/qiskit-terra/issues/6255)  
[Opflow takes the adjoint of an Operator if the expectation value is written as OperatorMeasurement #6254](https://github.com/Qiskit/qiskit-terra/issues/6254)  

In [1]:
import time
import numpy as np
import qiskit
from qiskit.opflow import X,Z,I
from qiskit.opflow.state_fns import StateFn, CircuitStateFn
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# copypaste niccolo's code
def QNKron(N,op1,op2,pos): 
    '''
    Tensor product operator (Qiskit Pauli operators)
    returns tensor product of op1,op2 on sites pos,pos+1 and identity on remaining sites

    N:number of sites
    op1,op2: Pauli operators on neighboring sites
    pos: site to insert op1
    '''
    temp=np.array([I]*(N))
    temp[pos]=op1 
    if pos!=(N-1):
        temp[pos+1]=op2
    mat=1
    for j in range(N):
        mat=mat^temp[j]
    return mat
def QHIsing(N,lam,p):
    '''
    Quantum Ising Hamiltonian (1D) with transverse field (Qiskit Pauli operators)
    
    N:number of sites 
    lam: transverse field)
    '''

    H=-QNKron(N,Z,Z,0)-lam*QNKron(N,X,I,0)-p*QNKron(N,Z,I,0)
    for i in range(1,N-1):
        H=H-QNKron(N,Z,Z,i)-lam*QNKron(N,X,I,i)-p*QNKron(N,Z,I,i)
    H=H-lam*QNKron(N,X,I,N-1)-p*QNKron(N,Z,I,N-1)
    return H

def Mag(N):
    sz=np.array([[1,0],[0,-1]])
    M=np.zeros((2**N,2**N))
    for i in range(N):
        M=M+NKron(N,sz,np.eye(2),i)
    return M/N

In [3]:
def sort_vals(vals):
    """ vals is (unsorted) dictionary of parameters from VQE ansatz circuit, this returns sorted values as list """
    indices = np.array([_.index for _ in vals])           # unordered list of indices from the ParameterVectorElement(Theta(INDEX))
    vals_sorted = np.array([vals[_] for _ in vals])       # unordered list of values (but same ordering as indices)
    return vals_sorted[np.argsort(indices)]

def init_vqe(vals):
    return qiskit.circuit.library.EfficientSU2(L, reps=3).assign_parameters(sort_vals(vals))

In [4]:
L = 6
VQE_vals = np.load(f'params_VQE_ising_N{L}.npy', allow_pickle=True).item()

lambdas = np.array([_ for _ in VQE_vals]) # list of lambda values (the items in the dictionary)
# note that Rike calls them gs

In [5]:
lambda0 = lambdas[0]

In [6]:
H = QHIsing(L,np.float32(lambda0),1e-4) # build Hamiltonian Op

In [7]:
state = init_vqe(VQE_vals[lambda0])
# state is technically a circuit, that prepares the ground state via VQE circuit
#state.draw() # uncomment to see, but is very long

In [8]:
mag = Z ^ Z ^ Z ^ Z ^ Z ^ Z

In [9]:
# the ~ is the adjoint, but also it turns the is_measurement attribute to True
~StateFn(mag)

OperatorStateFn(PauliOp(Pauli('ZZZZZZ'), coeff=1.0), coeff=1.0, is_measurement=True)

In [10]:
StateFn(state)

CircuitStateFn(<qiskit.circuit.library.n_local.efficient_su2.EfficientSU2 object at 0x7fd141507640>, coeff=1.0, is_measurement=False)

In [11]:
meas_outcome = ~StateFn(mag) @ StateFn(state)
meas_outcome.eval()

(0.9999460696468839+0j)

I also tried with the Hamiltonian but he didnt like that, I guess it has to do with ho Niccolo's code constructs the Hamiltonian and what StateFn can accept. Ideally we would construct the Hamiltonian with the routines described in the opflow tutorial